<a href="https://colab.research.google.com/github/rescobarcorrea/ArcGIS_server_managment/blob/main/ListMapServerConnectionsArcGISServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Get ArcGIS server MapServers, features and GDB conections**

In [ ]:
pip install retry xmltodict # Install stuff

In [ ]:
# Imports
import xmltodict
#from collections import OrderedDict
import json
import urllib
import urllib.request
import requests
#import collections

import numpy as np
from google.colab import files

from retry.api import retry_call
import logging
import ssl

In [ ]:
#Inputs
server="http://awsgis.fcnsw.com.au"

#port=6443
username="youruser"
password="yourpassword"
expiration=60
baseUrl = server+"/arcgis/admin/services"
#baseUrl = server+"/arcgis/rest/services/"
token_url=server+"/arcgis/tokens/generateToken"


#Logger details
logger = logging.getLogger('server_sync_testing_log.log')
logging.basicConfig(level=logging.INFO)

#Ingore SSL security

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [ ]:
#Functions
#Get your token here
#this funtion now wokring, note that you need to be admin of the server
class TokenError(Exception):
  'Invalid Token or busy server, retry again'
  pass

def catchTokenError (errormessage):
  if errormessage=='Invalid Token' or errormessage=='Invalid token, ClientID does not match.':
          raise TokenError('Invalid Token or busy server, retry again')
  else:
          #raise
          result=errormessage
          logger.error('not a token error')
  return result

def gentoken(url, username, password, expiration=60):
    #Get your token here
    #this funtion now wokring, note that you need to be admin of the server
    query_dict = {'username':   username,
                  'password':   password,
                  'expiration': str(expiration),
                  'client':     'requestip'}
    query_string = urllib.parse.urlencode(query_dict)
    query_string=query_string.encode('ascii')

    token=json.loads(urllib.request.urlopen(url + "?f=json", query_string,context=ctx).read())['token']
    
    if  not token:
        raise
        logging.error('token not generated')
        quit()
    else:
        return token
def getStatistics(service_url,token):
  #a function to read how much a service has been used
  #the service url has to be the one located in the admin folder
  #eg. https://<server>/arcgis/admin/services/<folder>/<service>.MapServer
  servicestatisticsurl=service_url+ "/statistics?f=json&token=" + token
  response=urllib.request.urlopen(servicestatisticsurl)
  statistics=json.load(response)
  #starttime=statistics['summary']['startTime']
  transactions=statistics['summary']['transactions']
  totalBusyTime=statistics['summary']['totalBusyTime']
  return transactions,totalBusyTime

# get list of features and gdb conections whithin a server
def GetServiceItems(service_url):
    metadata_url = '{0}/iteminfo/manifest/manifest.xml'.format(service_url)
    xml_data = requests.get(metadata_url,params={'token':token}).content
    data = dict(xmltodict.parse(xml_input=xml_data,encoding='UTF-8'))
    #print(data)
    service_items = []
    try:
      datasets = data['SVCManifest']['Databases']['SVCDatabase']['Datasets']['SVCDataset']
      mxd=data['SVCManifest']['Resources']['SVCResource']['OnPremisePath']
      name=data['SVCManifest']['Resources']['SVCResource']['Name']
      
      if isinstance(datasets,list):
          for ds in datasets:
            servicerow=[name,mxd,service_url,ds['Name'], ds['OnPremisePath'].split('\\')[-1]]
            service_items.append(servicerow)
      elif isinstance(datasets,OrderedDict):
          servicerow=[name,mxd,service_url,datasets['Name'], datasets['OnPremisePath'].split('\\')[-1]]
          service_items.append(servicerow)
      return service_items
    except:
      service_items=[[' -',' -',service_url,"manifest not found",' ']]
      return service_items


def getServicesList(baserestUrl,token,folderName=None):
  logger.warning('...requesting services manifesto')
  if not folderName:
    catalog=json.load(urllib.request.urlopen(baserestUrl + "/" + "?f=json&token=" + token ))
  else:
    catalog=json.load(urllib.request.urlopen(baserestUrl + "/" + folderName + "?f=json&token=" + token))
      
  try:
        errormessage=catalog['error']
        if errormessage['message']=='Invalid Token':
          raise TokenError('Invalid Token or busy server, retry again')
        else:
          raise
          result='error getting services list'
          logger.error(result)
  except KeyError:
        #print(TokenError.error)
        result=catalog
  return result
#Read manifesto


def getServerConections(server,token,tries=10,secondsoftry=5):
  baseUrl = server+"/arcgis/admin/services"
  catalog=retry_call(getServicesList,fargs=[baseUrl,token],tries=tries,delay=secondsoftry)
  
  services=catalog['services']
  itemis=[]
  counter=0
  for service in services:
      service_url=baseUrl + '/' + service['serviceName'] + '.' + service['type']
      #print ('----'+service['serviceName'], service['type'])
      #print(service_url)
      if service['type']=='MapServer':
        #items=[]
        print(service_url)
        items=retry_call(GetServiceItems,fargs=[service_url],tries=tries,delay=secondsoftry)
        itemis.append(items)
        print(items)
        
  if 'folders' in catalog:
    folders=catalog['folders']
    folders
    for folderName in folders:
      print(folderName)
      #token=gentoken(token_url,username,password,expiration)
      catalog=retry_call(getServicesList,fargs=[baseUrl,token],fkwargs={'folderName': folderName},tries=tries,delay=secondsoftry)
      services=catalog['services']
      for service in services:
                  counter+=1;
                  #if counter%100==0:
                    #token=gentoken(token_url,username,password,expiration)
                  service_url=baseUrl + '/' + folderName +'/'+ service['serviceName'] + '.' + service['type']
                  print ('----'+service['serviceName'], service['type'])
                  #print(service_url)
                  if service['type']=='MapServer':
                    items=retry_call(GetServiceItems,fargs=[service_url],tries=tries,delay=secondsoftry)
                    itemis.append(items)
                    print(items)
  return itemis
#end of functions

In [ ]:
token=gentoken(token_url,username,password,expiration)

In [ ]:
#To test for one server
token=gentoken(token_url,username,password,expiration)
serveritems=getServerConections(server,token)



http://awsgis.fcnsw.com.au/arcgis/admin/services/SampleWorldCities.MapServer


[['State Forest areas for fossicking under permit.mxd', 'P:\\RegionalMapComponents\\Corporate\\Maps\\WebServer\\Fossicking\\State Forest areas for fossicking under permit.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/SampleWorldCities.MapServer', 'FossickingExclusionLegalSF', 'GISS.FNSW.FossickingExclusionLegalSF'], ['State Forest areas for fossicking under permit.mxd', 'P:\\RegionalMapComponents\\Corporate\\Maps\\WebServer\\Fossicking\\State Forest areas for fossicking under permit.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/SampleWorldCities.MapServer', 'LegalSF', 'GISS.FNSW.LegalSF']]
CommunityServices


FCNSWcorp
----AOIsOnDemand MapServer
[[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/AOIsOnDemand.MapServer', 'manifest not found', ' ']]
----AvailableFirewoodCompartments MapServer
[[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/AvailableFirewoodCompartments.MapServer', 'manifest not found', ' ']]
----Beetest MapServer
[[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/Beetest.MapServer', 'manifest not found', ' ']]
----ComplianceTest MapServer
[['ComplianceTest.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\FCNSWcorp\\ComplianceTest.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/ComplianceTest.MapServer', 'Compliance', 'GISM.FNSW.Compliance'], ['ComplianceTest.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\FCNSWcorp\\ComplianceTest.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/ComplianceTest.MapServer', '

[['ForestHealth2017_20191218aws.mxd', 'C:\\Users\\mattn.FCNSW\\Documents\\ArcGIS\\2019\\ForMatthewNagel\\FeatureService\\ForestHealth2017_20191218aws.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/ForestHealth2017.MapServer', 'SketchHealthLine', 'gism.FNSW.SketchHealthLine'], ['ForestHealth2017_20191218aws.mxd', 'C:\\Users\\mattn.FCNSW\\Documents\\ArcGIS\\2019\\ForMatthewNagel\\FeatureService\\ForestHealth2017_20191218aws.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/ForestHealth2017.MapServer', 'SketchHealthFlightLines', 'gism.FNSW.SketchHealthFlightLines'], ['ForestHealth2017_20191218aws.mxd', 'C:\\Users\\mattn.FCNSW\\Documents\\ArcGIS\\2019\\ForMatthewNagel\\FeatureService\\ForestHealth2017_20191218aws.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/ForestHealth2017.MapServer', 'SketchHealthMpoint', 'gism.FNSW.SketchHealthMpoint'], ['ForestHealth2017_20191218aws.mxd', 'C:\\Users\\mattn.FCNSW\\Documents\\ArcGIS\\2019\\ForMatthe

[['Weeds.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\FCNSWcorp\\Weeds.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/Weeds.MapServer', 'WeedsExclusion', 'GISM.FNSW.WeedsExclusion'], ['Weeds.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\FCNSWcorp\\Weeds.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/Weeds.MapServer', 'WeedsTreatmentPoly', 'GISM.FNSW.WeedsTreatmentPoly'], ['Weeds.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\FCNSWcorp\\Weeds.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/Weeds.MapServer', 'WeedsTreatmentLine', 'GISM.FNSW.WeedsTreatmentLine'], ['Weeds.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\FCNSWcorp\\Weeds.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/Weeds.MapServer', 'WeedsTreatmentPoint', 'GISM.FNSW.WeedsTreatmentPoint'], ['Weeds.mxd', 'P:\\Region

...

----CachingControllers GPServer
----CachingTools GPServer
----PublishingTools GPServer
----ReportingTools GPServer
----SceneCachingControllers GPServer
----SceneCachingTools GPServer
----SpatialAnalysisTools GPServer
----SyncTools GPServer
Test
----Annotations MapServer


[[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/Annotations.MapServer', 'manifest not found', ' ']]
----BearingDistanceTest MapServer


[['zDev.mxd', 'C:\\MobileMXD\\zDev.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/BearingDistanceTest.MapServer', 'BearingDistanceLine', 'GISM.FNSW.BearingDistanceLine'], ['zDev.mxd', 'C:\\MobileMXD\\zDev.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/BearingDistanceTest.MapServer', 'HFD_Planpoints', 'gism.FNSW.HFD_Planpoints']]
----BiosecurityTest2 MapServer
[['BiosecurityTest.mxd', 'C:\\Users\\mattn.FCNSW\\Documents\\ArcGIS\\2019\\ForMatthewNagel\\FeatureService\\BiosecurityTest.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/BiosecurityTest2.MapServer', 'BiosecurityTreeTest', 'gism.FNSW.BiosecurityTreeTest'], ['BiosecurityTest.mxd', 'C:\\Users\\mattn.FCNSW\\Documents\\ArcGIS\\2019\\ForMatthewNagel\\FeatureService\\BiosecurityTest.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/BiosecurityTest2.MapServer', 'BiosecurityTreeInspectionTest', 'gism.FNSW.BiosecurityTreeInspectionTest']]
----CIFOAHarvestSupervision MapServer
[['CIFOAHar

[[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/HFDDumps.MapServer', 'manifest not found', ' ']]
----HFDtest MapServer
[['HFDContractorCIFOA.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\HFD\\HFDContractorCIFOA.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/HFDtest.MapServer', 'HFDTreesCoastal', 'GISM.FNSW.HFDTreesCoastal'], ['HFDContractorCIFOA.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\HFD\\HFDContractorCIFOA.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/HFDtest.MapServer', 'CIFOACompliance', 'GISM.FNSW.CIFOACompliance'], ['HFDContractorCIFOA.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMXDs\\AWSGISSVR\\HFD\\HFDContractorCIFOA.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/HFDtest.MapServer', 'HFDContractorNote', 'GISM.FNSW.HFDContractorNote'], ['HFDContractorCIFOA.mxd', 'P:\\RegionalMapComponents\\Corporate\\Projects\\ArcServerMX

[[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/Test/TestLGAs.MapServer', 'manifest not found', ' ']]
Utilities
----Geometry GeometryServer
----PrintingTools GPServer
----Search SearchServer
----Search_indexer GPServer


In [ ]:
#SaveCSV
filename='serverconectionsAWSGIS.csv'
with open(filename, 'w') as f:
  f.write('STUFF: ')


  for a in serveritems:
    for b in a:
      print (b)
      f.write (b[0]+','+b[1]+','+b[2]+','+b[3]+','+b[4]+'\n')

files.download(filename)

['State Forest areas for fossicking under permit.mxd', 'P:\\RegionalMapComponents\\Corporate\\Maps\\WebServer\\Fossicking\\State Forest areas for fossicking under permit.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/SampleWorldCities.MapServer', 'FossickingExclusionLegalSF', 'GISS.FNSW.FossickingExclusionLegalSF']
['State Forest areas for fossicking under permit.mxd', 'P:\\RegionalMapComponents\\Corporate\\Maps\\WebServer\\Fossicking\\State Forest areas for fossicking under permit.mxd', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/SampleWorldCities.MapServer', 'LegalSF', 'GISS.FNSW.LegalSF']
[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/AOIsOnDemand.MapServer', 'manifest not found', ' ']
[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/AvailableFirewoodCompartments.MapServer', 'manifest not found', ' ']
[' -', ' -', 'http://awsgis.fcnsw.com.au/arcgis/admin/services/FCNSWcorp/Beetest.MapServer', 'manifest not found', ' ']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Inspect each item
counter=0
for a in serveritems:
  for b in a:
    print (b)
